In [7]:
import mysql.connector
from pandas import DataFrame

from mysql.connector import errorcode
 
config = {
'user': 'db_gtown_2018',
'password': '***',
'port': '3306',
'host': 'nflnumbers.czuayagz62va.us-east-1.rds.amazonaws.com',
'database': 'db_nfl',
'raise_on_warnings': True,
}
 
try:
    cnx = mysql.connector.connect(**config)
    cursor = cnx.cursor()
    #Let's read all the rows in the table
    readContactPerson = """SELECT 
    FGXP,
    PBP.FKICKER,
    PNAME AS KICKER_NAME,
    DIST,
    QTR,
    MIN,
    TIMO AS TIMO_REMAIN,
    CASE
        WHEN GOOD = 'Y' THEN 1
        ELSE 0
    END AS SUCCESS,
    CASE
        WHEN DETAIL LIKE '%BLOCKED%' THEN 1
        ELSE 0
    END AS BLOCKED,
    CASE
        WHEN DPOS = 0 THEN 'UNDRAFTED'
        ELSE DPOS
    END AS DRAFT_POSITION,
    DIST - (100 - CASE
        WHEN YFOG = '' THEN '98'
        ELSE YFOG
    END) AS YDS_BEHIND_LOS,
    PTSO - PTSD AS POINT_DIFFERENTIAL,
    CASE
        WHEN
            QTR IN ('2' , '4') AND MIN <= 2
                AND (PTSO - PTSD) >= - 3
        THEN
            1
        ELSE 0
    END AS HIGH_PRESSURE,
    TEMP,
    HUMD,
    WSPD,
    WDIR,
    COND,
    SURF,
    DETAIL
FROM
    PBP
        LEFT OUTER JOIN
    PLAYER PLAYER ON PBP.FKICKER = PLAYER.PLAYER
        LEFT OUTER JOIN
    GAME ON PBP.GID = GAME.GID
WHERE
    TYPE = 'FGXP'
ORDER BY KICKER_NAME , PBP.GID;"""
    cursor.execute(readContactPerson)
    #specify the attributes that you want to display
    df = DataFrame(cursor.fetchall())    
    
    cnx.commit()
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cursor.close()
    cnx.close()

In [11]:
df.columns = ['FGXP','FKICKER','KICKER_NAME','DIST','QTR','MIN','TIMO_REMAIN','SUCCESS','BLOCKED','DRAFT_POSITION','YDS_BEHIND_LOS','POINT_DIFFERENTIAL','HIGH_PRESSURE','TEMP','HUMD','WSPD','WDIR','COND','SURF','DETAIL']

In [12]:
df.head(1)

,FGXP,FKICKER,KICKER_NAME,DIST,QTR,MIN,TIMO_REMAIN,SUCCESS,BLOCKED,DRAFT_POSITION,YDS_BEHIND_LOS,POINT_DIFFERENTIAL,HIGH_PRESSURE,TEMP,HUMD,WSPD,WDIR,COND,SURF,DETAIL
0,FG,AD-0800,A.Del Greco,38,2,7,3,1,0,UNDRAFTED,18.0,-7,0,73,78,10,S,Mostly Cloudy,AstroTurf,"(7:30) A.Del Greco 38 yard field goal is GOOD,..."


In [2]:
df[0].value_counts()

XP    22009
FG    18165
Name: 0, dtype: int64

In [6]:
df.groupby([0,7])[0,7].count()

0      7
0  7              
FG 0   3241   3241
   1  14924  14924
XP 0    438    438
   1  21571  21571

In [ ]:
df[7].value_counts()